<img src="https://www.th-koeln.de/img/logo.svg" style="float: right;" width="200">
<img src="https://www.femoz.de/assets/templates/femoz/images/logo-head-02.png" style="float: left;" width="200">
<p style="text-align:center;"><br><br>Project: FEMOZ<br>
Author of notebook: Florian Schmitt<br>
Date: 25.10.2022</p>

# Create sample data for water prediction

In [36]:
# load libraries
import pandas as pd
from sqlalchemy import MetaData, create_engine
import psycopg2
from random import random, uniform

In [37]:
# connect to db
def get_connection():
    try:
        return psycopg2.connect(
            database ='raw_data_db',
            user = 'fschmitt',
            password = 'jksHI93!)sewOl',
            host = '139.6.160.28',
            port = 5432,
        )
    except:
        return False

conn = get_connection()

if conn:
    # report status
    print('Connection to the PostgreSQL established successfully.')
else:
    # report status
    print('Connection to the PostgreSQL encountered and error.')

Connection to the PostgreSQL established successfully.


In [38]:
# Get table from raw_data_db
def get_data(table):
    try:
        # open connection
        conn = get_connection()

        # create a cursor using the connection object
        curr = conn.cursor()

        # execute the sql query
        curr.execute('SELECT * FROM ' + table + ';')

        # fetch colukmn headers
        column_names = [i[0] for i in curr.description]

        # fetch all the rows from the cursor
        data = curr.fetchall()

        # compile into dataframe
        df = pd.DataFrame.from_records(data, columns = column_names)

        # close connection
        conn.close()

        # report status
        print('Table successfully loaded.')

        # return result
        return df
    except:
        # report status
        print('An error ocurred.')


# define table to be loaded
table = 'admin_2'

# execute get function
admin_2 = get_data(table)

Table successfully loaded.


In [39]:
# create lists containing the setup data

# get list of ids from admin_2
id_list = list(admin_2.id_2)

# create list with different water supply types
#water = ['pipe', 'ground', 'surface']

# create list with year buckets
years = ['2011-2014', '2015-2018', '2019-2022']

# create list with value types
types = ['prediction', 'actual']

In [48]:
# create list to store results of for loop
results = []

for id in id_list:
    for year in years:
        for type in types:
            # generate random percentage values for water supply types
            value_ground = random()
            value_pipe = uniform(0,1-value_ground)
            value_surface = 1-value_ground-value_pipe

            # add to results list
            results.append([year, id, type, 'water', 'ground', round(value_ground, 4)])
            results.append([year, id, type, 'water', 'pipe', round(value_pipe, 4)])
            results.append([year, id, type, 'water', 'surface', round(value_surface, 4)])

            # generate random percentage values for education level types
            value_0_4 = random()
            value_5_7 = uniform(0,1-value_0_4)
            value_8_10 = uniform(0,1-value_5_7-value_0_4)
            value_11_more = 1-value_8_10-value_5_7-value_0_4

            # add to results list
            results.append([year, id, type, 'education', '0-4 years', round(value_0_4, 4)])
            results.append([year, id, type, 'education', '5-7 years', round(value_5_7, 4)])
            results.append([year, id, type, 'education', '8-10 years', round(value_8_10, 4)])
            results.append([year, id, type, 'education', '11+ years', round(value_11_more, 4)])

In [49]:
# write result list into csv file
pd.DataFrame(results).to_csv('sample_data.csv', index = False, header = ['year', 'id_2', 'value_type', 'measure', 'attribute', 'value'])